# Do Video Games Cause Aggression And What Is Their Impact At All?

This is a vast field to research and a lot of aspects of our life to investigate. In my research, I will take a look at the main of them (in my opinion). My possibilities will be limited by the quality of open data.

In folder `data` you can find all datasets related to the current topic that I have found. Some of them are parsed by myself. I already filtered what datasets can be useful in my work and will use them here.

*P.S. If you read this in jupyter notebook you can interact with every plot here. That is, even if plot looks a litttle bit messy, you can easely understand it using tools provided by `plotly`*

In [115]:
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import plotly.express as px
from plotly.offline import plot
import geoplotlib as geoplt
import plotly.graph_objects as go

import pycountry

In [116]:
state2abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'St. Croix': 'St. Croix, US Virgin Islands',
}

abbrev2state = dict(map(reversed, state2abbrev.items()))

In [117]:
DATA_CRIME = 'data/crime/'
DATA_GAMES = 'data/games/'
DATA_OTHER = 'data/other/'
DATA_SCHOOL= 'data/school/'
DATA_WEAPON= 'data/weapon/'

## Games statistics

In [118]:
# ps4_sales   = pd.read_csv(DATA_GAMES + 'PS4_GamesSales.csv', encoding = "ISO-8859-1")
# xbox_sales  = pd.read_csv(DATA_GAMES + 'XboxOne_GameSales.csv', encoding = "ISO-8859-1")
# steam       = pd.read_csv(DATA_GAMES + 'steam.csv')
# steam_description_data = pd.read_csv(DATA_GAMES + 'steam_description_data.csv')

Firstly, let's observe data about game sales by the world. Each dataset contains info about 100 most sold games in some year (one file - one year) from 2005 to 2018 including.

In [119]:
dfs = []
for year in range(2005, 2019):
    temp_df = pd.read_csv(f"{DATA_GAMES}{year}.csv").drop(['Unnamed: 0'], axis=1)
    new_index = pd.MultiIndex.from_product([[year], temp_df.index], names=['Year', 'Index'])
    temp_df.reset_index(inplace=True)
    temp_df.set_index(new_index, inplace=True)
    dfs.append(temp_df)
    
years_df = pd.concat(dfs).drop(['index'], axis=1)

In [120]:
years_df.head()

Game Platform  \
Year Index                                                    
2005 0                                  Nintendogs       DS   
     1                               Mario Kart DS       DS   
     2      Grand Theft Auto: Liberty City Stories      PSP   
     3                           Super Mario 64 DS       DS   
     4                 Animal Crossing: Wild World       DS   

                       Publisher       Genre   Global        USA     Europe  \
Year Index                                                                    
2005 0                  Nintendo  Simulation  4722694  1485047.0  3151987.0   
     1                  Nintendo      Racing  2775296   846418.0   963501.0   
     2      Take-Two Interactive      Action  2132118   657231.0   562851.0   
     3                  Nintendo    Platform  2029244   659007.0   428719.0   
     4                  Nintendo  Simulation  1699892   242645.0  1169228.0   

                Japan       UK  Germany   France  
Year Index                                        
2005 0      1088738.0      NaN      NaN  34808.0  
     1       853488.0  53839.0  41962.0  44087.0  
     2            NaN      NaN      NaN      NaN  
     3       341914.0      NaN      NaN      NaN  
     4      1402362.0      NaN      NaN      NaN

In [121]:
# returns dataframe that represents how much games in some genre were sold in some year
def reduce_data_by_genres(data, only_usa=False):
    genres_list = data['Genre'].unique()
    if only_usa:
        data['USA'] = pd.to_numeric(data['USA'])
    else:
        data['Global'] = pd.to_numeric(data['Global'])
    reduced_data_by_year = pd.DataFrame()
    reduced_data_by_year['Year'] = data.index.get_level_values(0).unique()
    sales = []
    genres = {genre: [] for genre in genres_list}

    for i, row in reduced_data_by_year.iterrows():
        if only_usa:
            sales.append(data[data.index.get_level_values(0) == row['Year']]['USA'].sum())
        else:
            sales.append(data[data.index.get_level_values(0) == row['Year']]['Global'].sum())
        for genre in genres:
            if only_usa:
                genres[genre].append(
                    data[(data.index.get_level_values(0) == row['Year'])
                        & (data['Genre'] == genre)]['USA'].sum()
                )            
            else:
                genres[genre].append(
                    data[(data.index.get_level_values(0) == row['Year'])
                        & (data['Genre'] == genre)]['Global'].sum()
                )

    reduced_data_by_year['Sales'] = pd.Series(sales)
    for genre in genres:
        reduced_data_by_year[genre] = pd.Series(genres[genre])

    return reduced_data_by_year

In [122]:
# returns dataframe that represents how much games were sold in some year in some country
def reduce_data_by_countries(data):
    genres_list = data['Genre'].unique()
    
    reduced_data_by_country = pd.DataFrame()
    reduced_data_by_country['Year'] = data.index.get_level_values(0).unique()
    
    country_sales = {
        'USA': [], 'Europe': [], 'Japan': [], 'UK': [], 'Germany': [], 'France': [], 'Global': []
    }
    for country in country_sales:
        data[country] = pd.to_numeric(data[country])
        for year in range(2005, 2019):
            country_sales[country].append(data[data.index.get_level_values(0) == year][country].sum())
        reduced_data_by_country[country] = country_sales[country]
    
    return reduced_data_by_country

In [123]:
def hist_by_genres(data):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='Total Sales',
                               x=data['Year'], y=data["Sales"],
                               histfunc='sum', xbins={'size':1} ))
    for genre in data.columns[2:]:
        fig.add_trace(go.Histogram(name=genre,
                                   x=data['Year'], y=data[genre],
                                   histfunc='sum', xbins={'size':1} ))

    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.show()

In [124]:
def hist_by_years(data, opacity=.75):
    fig = go.Figure()

    for feature in data.columns[1:]:
        fig.add_trace(go.Histogram(name=feature,
                                   x=data['Year'], y=data[feature],
                                   histfunc='sum', xbins={'size':1} ))

    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=opacity)
    fig.show()

In [125]:
reduced_years_df = reduce_data_by_genres(years_df)
reduced_years_df.head(3)

,Year,Sales,Simulation,Racing,Action,Platform,Misc,Puzzle,Sports,Role-Playing,Strategy,Shooter,Fighting,Adventure,Action-Adventure,Party,Sandbox,Music
0,2005,43366369,7476416,8024644,4317383,4251008,3515759,2582587,5144933,2347875,981389,2447543,1174596,1102236,0,0,0,0
1,2006,97437003,14094283,6596768,6466547,12844669,16732260,2562725,9423917,12562408,935713,9975390,2713416,2528907,0,0,0,0
2,2007,180506379,14805742,12442024,6835103,19330718,37194517,2704444,25984112,21979845,0,25393966,2239964,11595944,0,0,0,0


In [126]:
hist_by_genres(reduced_years_df)

From this chart, we can find that the genre "sports" (the talk is about Wii Sports mostly, I suggest) was very popular until 2012. "Action" genre came instead of it. Shooters were popular almost overall time. The last two genres have a lot of killings and violence scenes inside of them. Most psychologists say that such possibility allows people to expel their aggression in games and to not do this on their relatives or friends. That's good, right? If it's true, then yes, of course. But there are some people who state that such behavior in games wakes up aggression in real life. In my opinion, if a person has good mental health, he/she understands where is a game, and where real life. However, these thoughts are not facts. Now, we will take a look at numbers and try to figure out some conclusions about real impact of video games.

In [127]:
reduced_data_by_countries = reduce_data_by_countries(years_df)
reduced_data_by_countries.head(2)

,Year,USA,Europe,Japan,UK,Germany,France,Global
0,2005,15306467.0,11276528.0,12120548.0,53839.0,41962.0,78895.0,43366369
1,2006,29882087.0,18806256.0,32657962.0,219361.0,277816.0,304748.0,97437003


In [128]:
hist_by_years(reduced_data_by_countries)

# Bad (or good?) impact of games

The following data is gathered surveying  students in the high schools of the State of Connecticut anonymously. It was made for research with a similar topic as mine.

So, we have table in the following format

|          |             | N  | %  | N play video games ever | % play video games ever |  X2 |  p  |
|----------|-------------|----|----|-------------------------|-------------------------|-----|-----|
| Var Name | Var Value 1 | n1 | %1 |          number1        |         percent1        | val |  p  |
| Var Name | Var Value 2 | n2 | %2 |          number2        |         percent2        | nan | nan |
| Var Name | Var Value 3 | n3 | %3 |          number3        |         percent3        | nan | nan |

In column `N` there are numbers of students that belong to each category (`var value n`), and `%` means percent that corresponds to that number. It is quite obvious as well as the next column. `% play video games ever` can be calculated as $ \frac{N play Video Games Ever}{N} $ (not total number). To `X2` and `p`, we'll return in the farther cells. We also see that the columns are duplicated. So, the first copy (where column names have `.1` in the end) stands for boys and the second one for girls respectively. The first copy is just the sum of the others.

In [129]:
students_survey = pd.read_excel(DATA_SCHOOL + 'students_survey.xlsx')
students_survey

,Variable Name,Variable Value,N,%,N play video games ever,% play video games ever,X2,p,N.1,%.1,N play video games ever.1,% play video games ever.1,X2.1,p.1,N.2,%.2,N play video games ever.2,% play video games ever.2,X2.2,p.2
0,Grade average,A's and B's,2319,59.11,1129,48.68,15.48,0.0004,941,52.05,736,78.21,6.7393,0.0344,1361,65.50,383,28.14,2.5883,0.2741
1,Grade average,C's,1157,29.49,634,54.80,NaN,NaN,594,32.85,450,75.76,NaN,NaN,549,26.42,173,31.51,NaN,NaN
2,Grade average,D's and F's,447,11.39,249,55.70,NaN,NaN,273,15.10,193,70.70,NaN,NaN,168,8.08,53,31.55,NaN,NaN
3,Extra-curricular activities,Yes,3056,75.87,1557,50.95,0.43,0.5104,1382,74.91,1057,76.48,0.0870,0.768,1642,76.76,477,29.05,0.0980,0.7543
4,Extra-curricular activities,No,972,24.13,507,52.16,NaN,NaN,463,25.09,351,75.81,NaN,NaN,497,23.24,148,29.78,NaN,NaN
5,"Smoking, lifetime",Never,2441,62.41,1328,54.40,34.34,<0.0001,1146,64.53,912,79.58,29.9100,<0.0001,1268,60.53,398,31.39,12.4200,0.002
6,"Smoking, lifetime",Occasionally,949,24.26,415,43.73,NaN,NaN,397,22.35,284,71.54,NaN,NaN,546,26.06,127,23.26,NaN,NaN
7,"Smoking, lifetime",Regularly,521,13.32,246,47.22,NaN,NaN,233,13.12,155,66.52,NaN,NaN,281,13.41,85,30.25,NaN,NaN
8,"Marijuana, lifetime",Yes,1476,39.07,698,47.29,11.72,0.0006,689,40.39,487,70.68,16.8500,<0.0001,772,25.78,199,25.78,7.1100,0.0077
9,"Marijuana, lifetime",No,2302,60.93,1220,53.00,NaN,NaN,1017,59.61,807,79.35,NaN,NaN,1259,61.97,394,31.32,NaN,NaN


In [130]:
def value_picker(data, variable_name=None, variable_value=None, column=None):
    if not (variable_name or variable_value or column):
        return data
    if variable_name and variable_value and column:
        return data[ (data['Variable Name'] == variable_name)
                & (data['Variable Value'] == variable_value) ]\
                [column]
    if variable_name and column:
        return data[data['Variable Name'] == variable_name][column]

Authors of that research also have done a chi-squared test with the Null Hypothesis that computer games do not influence certain the aspect of students' life (`variable name`). Let's don't waste time repeating the test for all categories but do only one. For other categories will use already precalculated values of `chi-squared` and `p` which are stored in columns `X2` and `pp` respectively.

In [131]:
res = stats.chisquare(value_picker(students_survey, 'Smoking, lifetime', column='%'),
                value_picker(students_survey, 'Smoking, lifetime', column='% play video games ever'))
print('Chi-squred:', res[0])
print('p-value:', res[1])
print('Reject' if res[1] < .05 else 'Accept', "Null Hypothesis that computer games do not influence certain the aspect of students'")

Chi-squred: 34.18543976091471
p-value: 3.773341179565858e-08
Reject Null Hypothesis that computer games do not influence certain the aspect of students'


That is, we now know that among those students who never smoke, there are a lot of gamers. Investigating other categories that we have, we can see that among boys, games are associated with never having used marijuana, high caffeine consumption, and higher grades. This not necessary means that games make boys smarter and teach not to take drugs; it also can demonstrate that most such boys play video games. For girls situation is a bit similar - games are associated with occasional smoking, never having used marijuana, never having a sip of alcohol, high caffeine use, no history of depression, getting into serious fights, and carrying a weapon. Shortly speaking, games are associated with no bad habits for health, but also with problematic, maybe aggressive behavior.

# About crimes

We have already observed data about the popularity of different games' genres in the world. Let's take a look at school shooting statistics. Unfortunately (actually, fortunately), I have found data on school shootings in the USA and Sweden only.

In [132]:
usa_shootings = pd.read_csv(DATA_SCHOOL + 'K-12 SSDB (Public) - K-12 SSDB (Public) Linked.csv', skiprows=1)
sweden_shootings = pd.read_csv(DATA_SCHOOL + 'school_fire_cases_1998_2014.csv')

# data4 = pd.read_csv(DATA_SCHOOL + 'pah_wikp_combo.csv')
# data6 = pd.read_csv(DATA_SCHOOL + 'school_scores.csv') 
# data7 = pd.read_csv(DATA_SCHOOL + 'simplified_municipality_indicators.csv')

# data1 = pd.read_excel(DATA_SCHOOL + 'AAA-Mass Shooting Attacks (3 or more injured and-or killed) 10-12-2018.xlsx')
# data2 = pd.read_excel(DATA_SCHOOL + 'ASE- Potential Active Shooter Events 9-24-2018 BINARY.xlsx')
# data8 = pd.read_excel(DATA_SCHOOL + 'SSS-School Shooting Database.xlsx')

In [133]:
usa_shootings.head()

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Firearm Type,Number of Shooters,Shooter Name,Shooter Age,Shooter Gender,Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N)
0,1/5/1970,Hine Junior High School,Washington,DC,3.0,1,0.0,1,Male,Student,...,Handgun,1.0,Minor,15.0,M,Unknown,Unknown,Student,Y,N
1,1/5/1970,Sousa Junior High,Washington,DC,3.0,0,1.0,1,Male,Student,...,Handgun,1.0,Minor,NaN,M,Unknown,Unknown,Student,N,N
2,1/5/1970,Unnamed High School,Washington,DC,2.0,0,0.0,0,No Victims,No Victims,...,Handgun,1.0,Unknown,NaN,M,Unknown,Unknown,Student,Y,N
3,2/6/1970,John F. Kennedy High School,Cleveland,OH,2.0,0,1.0,1,Male,Student,...,Handgun,1.0,Gertis J. Perry,18.0,M,Unknown,Not Hispanic or Latino,Student,N,N
4,3/23/1970,David Starr Jordan High School,Long Beach,CA,2.0,0,2.0,2,Male,Multiple Victims,...,Handgun,2.0,"Obidde Cowart, Lavaughn Hunter",NaN,M,Unknown,Not Hispanic or Latino,Other Staff,Y,N


In [134]:
usa_shootings_years = pd.DataFrame()
usa_shootings['Year'] = [date.year for date in pd.to_datetime(usa_shootings['Date'])]
usa_shootings_years['Year'] = usa_shootings['Year'].unique()

cases = []
killed = []
wounded = []
for year in usa_shootings_years['Year']:
    cases.append(len(usa_shootings[usa_shootings['Year'] == year]))
    killed.append(usa_shootings[usa_shootings['Year'] == year]['Killed (includes shooter)'].sum())
    wounded.append(usa_shootings[usa_shootings['Year'] == year]['Wounded'].sum())

usa_shootings_years['Cases'] = pd.Series(cases)
usa_shootings_years['Wounded'] = pd.Series(wounded)
usa_shootings_years['Killed'] = pd.Series(killed)
usa_shootings_years.head()

,Year,Cases,Wounded,Killed
0,1970,20,24.0,8
1,1971,21,19.0,9
2,1972,18,22.0,6
3,1973,18,25.0,6
4,1974,16,23.0,12


In [135]:
hist_by_years(usa_shootings_years)

In general, we have enormous growth in cases of school shootings starting from 2010. It's (if we can say so) good that the number of killed doesn't grow so fast, but even 5 persons (1990, 2011) is a big number of humans' lives.

In [136]:
reduced_years_df.head()

,Year,Sales,Simulation,Racing,Action,Platform,Misc,Puzzle,Sports,Role-Playing,Strategy,Shooter,Fighting,Adventure,Action-Adventure,Party,Sandbox,Music
0,2005,43366369,7476416,8024644,4317383,4251008,3515759,2582587,5144933,2347875,981389,2447543,1174596,1102236,0,0,0,0
1,2006,97437003,14094283,6596768,6466547,12844669,16732260,2562725,9423917,12562408,935713,9975390,2713416,2528907,0,0,0,0
2,2007,180506379,14805742,12442024,6835103,19330718,37194517,2704444,25984112,21979845,0,25393966,2239964,11595944,0,0,0,0
3,2008,267489984,17255708,20383898,32045076,19273193,56066546,0,53564286,20623787,0,29044207,8956017,10277266,0,0,0,0
4,2009,240385516,5254911,18837964,23375096,17129336,16547338,908463,71947924,26287675,1628033,35644209,11704858,11119709,0,0,0,0


In [137]:
reduced_years_df_usa = reduce_data_by_genres(years_df, True)

In [183]:
games_solds = pd.DataFrame()
games_solds['Year'] = reduced_years_df_usa['Year']
games_solds['Games Sold'] = reduced_years_df_usa['Shooter']\
                        + reduced_years_df_usa['Action']\
                        + reduced_years_df_usa['Fighting']
games_solds.head()

,Year,Games Sold
0,2005,3917593.0
1,2006,9630080.0
2,2007,14972552.0
3,2008,28730948.0
4,2009,30000530.0


In [185]:
games_solds['Shootings Cases'] = pd.Series(usa_shootings_years[(usa_shootings_years['Year'] >= 2005)\
                       & (usa_shootings_years['Year'] <= 2018)]['Cases'].values)
# games_solds['Shootings'] /= max(games_solds['Shootings'])
# games_solds['Games Sold'] /= max(games_solds['Games Sold'])

fig = px.scatter(games_solds, x="Games Sold", y="Shootings Cases", trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

C:\Users\3naza\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



C:\Users\3naza\Anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=14



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Shootings Cases   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     6.211
Date:                Mon, 09 Dec 2019   Prob (F-statistic):             0.0283
Time:                        16:57:03   Log-Likelihood:                -61.341
No. Observations:                  14   AIC:                             126.7
Df Residuals:                      12   BIC:                             128.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         75.9313     14.610      5.197      0.000      44.099     107.763
Games Sold  -1.27e-06    5.1e-07     -2.492      0.028   -2.38e-06    -1.6e-07
==============================================================================
Omnibus:                       24.878   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.024
Skew:                           2.304   Prob(JB):                     4.98e-07
Kurtosis:                       8.341   Cond. No.                     7.50e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.5e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Here it is, for years from 2005 to 2018 we have such a situation that the more games where you should kill are bought, the less school shooting rate. The coefficient of correlation is `-0.4633` with constant `0.6603`. It looks like games really help you to expel your aggression in virtual world.<br>
I also remind that talk is about the USA only.

In [140]:
sweden_shootings.head(3)

,Municipality,Cases,Population,Year
0,Ale,0,25292,1998
1,Alingsås,0,34930,1998
2,Alvesta,0,19171,1998


In [141]:
sweden_shootings_years = pd.DataFrame()
sweden_shootings_years['Year'] = sweden_shootings['Year'].unique()

cases = []
population = []
for year in sweden_shootings_years['Year']:
    cases.append(sweden_shootings[sweden_shootings['Year'] == year]['Cases'].mean())
    population.append(sweden_shootings[sweden_shootings['Year'] == year]['Population'].mean())

sweden_shootings_years['Population'] = pd.Series(population) / max(population)
sweden_shootings_years['Cases'] = pd.Series(cases)
sweden_shootings_years.head()

,Year,Population,Cases
0,1998,0.913892,0.416667
1,1999,0.912257,0.550173
2,2000,0.914456,0.550173
3,2001,0.917167,0.550173
4,2002,0.919131,0.480969


In [142]:
hist_by_years(sweden_shootings_years)

For Sweden data is in different formats and isn't so informative as for the USA. On histogram `Cases` means the average number of cases in all cities in some year. `Population` is the sum of populations of each city scaled to the range [0, 1]. There is some growth from 2005 to 2010. This situation differs from situation of the USA.

In [143]:
# data2015 = pd.read_csv(DATA_OTHER + '2015.csv')
# data2016 = pd.read_csv(DATA_OTHER + '2016.csv')
# data2017 = pd.read_csv(DATA_OTHER + '2017.csv')
# data2017_1 = pd.read_csv(DATA_OTHER + '2017(1).csv')
# data2018 = pd.read_csv(DATA_OTHER + '2018.csv')
# data2019 = pd.read_csv(DATA_OTHER + '2019.csv')

I found this dataset accidentally and decided to observe it. Who knows, maybe a failure in a virtual game makes somebody hate real one?

In [144]:
suicides = pd.read_csv(DATA_OTHER + 'master.csv')

In [145]:
suicides.head(3)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X


In [146]:
suicides_years = pd.DataFrame()
suicides_years['Year'] = suicides['year'].unique()
suicides_years_top = pd.DataFrame()
suicides_years_top['Year'] = suicides['year'].unique()

countries = suicides['country'].unique()
suicides_no = {country: [] for country in countries}
for country in countries:
    for year in suicides_years['Year']:
        suicides_no[country].append(
            suicides[(suicides['country'] == country)
                    & (suicides['year'] == year)
                    ]['suicides_no'].sum())

suicides_no = [sorted(suicides_no.items(), key=lambda x: np.mean(x[1]), reverse=True)][0]

for country in suicides_no:
    suicides_years[country[0]] = pd.Series(country[1])

for country in suicides_no[:10]:
    suicides_years_top[country[0]] = pd.Series(country[1])
suicides_years_top.head()

,Year,Russian Federation,United States,Japan,France,Ukraine,Germany,Republic of Korea,Brazil,Poland,United Kingdom
0,1987,0,30783,23663,12161,10050,0,3301,4672,0,4594
1,1988,0,30388,22676,11599,9751,0,2949,4466,0,4971
2,1989,37921,30218,21013,11715,10887,0,3023,4463,0,4361
3,1992,45923,30471,20756,11644,11666,13458,3533,5207,5713,4628
4,1993,55846,31084,20353,12251,12469,12690,4124,5508,5624,4462


In [147]:
hist_by_years(suicides_years_top)

There are a lot of suicide cases in Ukraine and Russia in period from 1990 to 2010. Likely, it is related to "bad 90-s".

The next thing is the correlation between the suicide rate in the world and the games' sold. Yes, it depends on many factors (for example, we don't know whether some of those who committed suicide played computer games) and if there will be some correlation, it's likely random.

In [148]:
games_solds2 = pd.DataFrame()
games_solds2['Year'] = reduced_years_df_usa[reduced_years_df_usa['Year'] <= 2015]['Year']
games_solds2['Solds'] = reduced_years_df_usa[reduced_years_df_usa['Year'] <= 2015]['Shooter']\
                        + reduced_years_df_usa[reduced_years_df_usa['Year'] <= 2015]['Action']\
                        + reduced_years_df_usa[reduced_years_df_usa['Year'] <= 2015]['Fighting']
games_solds2

,Year,Solds
0,2005,3917593.0
1,2006,9630080.0
2,2007,14972552.0
3,2008,28730948.0
4,2009,30000530.0
5,2010,36407494.0
6,2011,41952406.0
7,2012,41617496.0
8,2013,36125207.0
9,2014,29735040.0


In [149]:
def shorten_data(suicides_years, years=(2005, 2015)):
    suicides_g = suicides_years[(suicides_years['Year'] >= years[0])
                         & (suicides_years['Year'] <= years[1])]
    s = []
    for year in suicides_g['Year']:
        val = []
        for country in suicides_years.columns[1:]:
            new_val = suicides_years[suicides_years['Year'] == year][country].values[0]
            if new_val:
                val.append(new_val)
        s.append(np.mean(val))

    suicides_g.drop(suicides_g.columns[1:], axis=1, inplace=True)
    suicides_g['Sum'] = pd.Series(s, index=suicides_g.index)
    return suicides_g

In [150]:
suicides_g = shorten_data(suicides_years)
suicides_g

C:\Users\3naza\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Year,Sum
16,2005,2858.231707
17,2006,2917.012500
18,2007,2778.666667
19,2008,2906.753086
20,2009,2831.244186
21,2010,2743.701149
26,2012,2913.417722
27,2013,2936.828947
28,2014,2934.000000
29,2015,3338.360656


In [151]:
games_solds2['Suicides'] = pd.Series(suicides_g['Sum'].values)
games_solds2['Suicides'] /= max(games_solds2['Suicides'])
games_solds2['Solds'] /= max(games_solds2['Solds'])

fig = px.scatter(games_solds2, x="Solds", y="Suicides", color='Year', trendline="lowess",
                color_continuous_scale=px.colors.sequential.Plotly3)
fig.show()

Interesting result.

In [152]:
suicides_countries = pd.DataFrame()
codes = []
countries = []
values = []
for country in suicides_years.columns[1:]:
    try:
        codes.append(pycountry.countries.get(name=country).alpha_3)
        countries.append(country)
        values.append(suicides_years[suicides_years[country] != 0][country].mean())
    except:
        pass

suicides_countries['Country Code'] = pd.Series(codes)
suicides_countries['Country'] = pd.Series(countries)
suicides_countries['Suicides'] = pd.Series(values)
suicides_countries.head()

,Country Code,Country,Suicides
0,RUS,Russian Federation,44805.259259
1,USA,United States,33355.258065
2,JPN,Japan,26029.096774
3,FRA,France,10970.900000
4,UKR,Ukraine,11426.785714


In [153]:
def plot_world(locations, z, colorscale, text, colorbar_title='', title_text=''):
    fig = go.Figure(data=go.Choropleth(
        locations=locations,
        z = z,
        colorscale = colorscale,
        autocolorscale=False,
        text=text,
        marker_line_color='white',
        colorbar_title = colorbar_title,
    ))

    fig.update_layout(
        title_text = title_text,
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
    )

    fig.show()

In [154]:
plot_world(suicides_countries['Country Code'], suicides_countries['Suicides'],
          'reds', suicides_countries['Country'], '', 'Suicides Numbers from 1985 to 2005')

The very first thing I have noticed is that Crimea is marked as the Russian Federation, and this is totally unacceptable. I will start the corresponding discussion on plotly's GitHub as soon as I finish this research.

In [155]:
gdp_years = pd.DataFrame()
gdp_years['Year'] = suicides['year'].unique()

countries = suicides['country'].unique()
gdps = {country: [] for country in countries}
for country in countries:
    for year in gdp_years['Year']:
        gdps[country].append(
            suicides[(suicides['country'] == country)
                    & (suicides['year'] == year)
                    ]['gdp_per_capita ($)'].sum())

for country in suicides_years.columns[1:]:
    gdp_years[country] = pd.Series(gdps[country])
gdp_years.head()

,Year,Russian Federation,United States,Japan,France,Ukraine,Germany,Republic of Korea,Brazil,Poland,...,Kiribati,Cabo Verde,Grenada,Oman,Macau,Maldives,Antigua and Barbuda,San Marino,Dominica,Saint Kitts and Nevis
0,1987,0,259572,265644,215988,16236,0,46008,28728,0,...,0,0,0,0,0,0,67140,0,0,0
1,1988,0,277236,320244,234408,18852,0,61068,31536,0,...,0,0,32484,0,0,0,80316,0,0,0
2,1989,44880,295848,316668,234696,20784,0,74604,39720,0,...,0,0,39216,0,0,0,88884,0,0,0
3,1992,39996,333120,399840,313632,18300,334656,104052,35088,31800,...,9108,0,44556,0,0,0,97044,0,0,70416
4,1993,37920,346692,453984,294252,16188,323244,113748,37632,32196,...,8820,0,43584,0,0,0,101436,0,0,0


In [156]:
hist_by_years(gdp_years)

In [157]:
suicides_shorten = shorten_data(suicides_years, (2005, 2014))
suicides_shorten['GDP'] = shorten_data(gdp_years, (2005, 2014))['Sum']

C:\Users\3naza\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [158]:
suicides_shorten['Sum'] /= max(suicides_shorten['Sum'])
suicides_shorten['GDP'] /= max(suicides_shorten['GDP'])

fig = px.scatter(suicides_shorten, x="GDP", y="Sum", color='Year', trendline="ols",
                color_continuous_scale=px.colors.sequential.matter)
fig.show()

results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



C:\Users\3naza\Anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=10



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Sum   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.9150
Date:                Mon, 09 Dec 2019   Prob (F-statistic):              0.367
Time:                        14:07:56   Log-Likelihood:                 23.959
No. Observations:                  10   AIC:                            -43.92
Df Residuals:                       8   BIC:                            -43.31
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9028      0.075     12.080      0.000       0.730       1.075
GDP            0.0809      0.085      0.957      0.367      -0.114       0.276
==============================================================================
Omnibus:                        1.809   Durbin-Watson:                   0.915
Prob(Omnibus):                  0.405   Jarque-Bera (JB):                1.099
Skew:                          -0.549   Prob(JB):                        0.577
Kurtosis:                       1.803   Cond. No.                         19.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The correlation isn't strong as we take average values over all countries that we have, so specifics of some countries are lost. For example, countries with extremely low GDP don't have enough information about suicide and/or possibilities to commit it.

In [159]:
ages = sorted(suicides['age'].unique(),
              key=lambda x: int(x.split()[0].replace('+', '00').replace('-', '')))
indeces = pd.MultiIndex.from_product([['Male', 'Female'], ages], names=['Sex', 'Age'])
suicides_by_sex_age = pd.DataFrame(np.nan, index = indeces, columns=['Suicides Number'])

for sex, age in zip(suicides_by_sex_age.index.get_level_values(0),
                    suicides_by_sex_age.index.get_level_values(1)):
    suicides_by_sex_age.loc[sex, age] = suicides[(suicides['sex'] == sex.lower())
                                 & (suicides['age'] == age)]\
                                   ['suicides_no'].sum()

mages = '|M_'.join(ages).split('|')
mages[0] = 'M_' + mages[0]
fages = '|F_'.join(ages).split('|')
fages[0] = 'F_' + fages[0]
male_n = suicides_by_sex_age.loc['Male'].sum() / 1_000_000
female_n = suicides_by_sex_age.loc['Female'].sum() / 1_000_000

fig =go.Figure(go.Sunburst(
    labels = [*['Male', 'Female'],
              *mages,
              *fages],
    parents = [*['', ''],
               *['Male' for _ in range(len(ages))], *['Female' for _ in range(len(ages))]],
    values = [ *male_n, *female_n,
              *suicides_by_sex_age['Suicides Number'].values],
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()

If you have ever been interested who and in what age commits suicide, here it is.

Unlikely such social "parameters" as the generosity of confidence in the national government are related to video games but they can have an impact on the suicide rate. This is not related to my topic directly, but it is a socially important question.<br> Since all attributes are scaled to the range [0, 1], I will take the average value to simplify data. However, I will replace `Perceptions of corruption` with `1 - Perceptions of corruption`, since the smaller this value is, the better.

In [160]:
social = pd.read_csv(DATA_OTHER + 'Original_2017_full.csv',
                         usecols=['country', 'Freedom to make life choices',
                                 'Generosity', 'Perceptions of corruption',
                                 'Confidence in national government', 'year'])
social.head()

,country,year,Freedom to make life choices,Generosity,Perceptions of corruption,Confidence in national government
0,Afghanistan,2008,0.718114,0.181819,0.881686,0.612072
1,Afghanistan,2009,0.678896,0.203614,0.850035,0.611545
2,Afghanistan,2010,0.600127,0.137630,0.706766,0.299357
3,Afghanistan,2011,0.495901,0.175329,0.731109,0.307386
4,Afghanistan,2012,0.530935,0.247159,0.775620,0.435440


In [161]:
social.describe()

,year,Freedom to make life choices,Generosity,Perceptions of corruption,Confidence in national government
count,1562.000000,1533.000000,1482.000000,1472.000000,1401.000000
mean,2011.820743,0.728975,0.000079,0.753622,0.480207
std,3.419787,0.145408,0.164202,0.185538,0.190724
min,2005.000000,0.257534,-0.322952,0.035198,0.068769
25%,2009.000000,0.633754,-0.114313,0.697359,0.334732
50%,2012.000000,0.748014,-0.022638,0.808115,0.463137
75%,2015.000000,0.843628,0.094649,0.880089,0.610723
max,2017.000000,0.985178,0.677773,0.983276,0.993604


In [162]:
social.dropna(inplace=True)

In [163]:
years = social['year'].unique()
countries = social['country'].unique()
social_years = pd.DataFrame()
social_years['Year'] = pd.Series(years)

social_ind = {country: [] for country in countries}
for country in countries:
    for year in years:
        ind = []
        for d in social.columns[2:]:
            if d == 'Perceptions of corruption':
                val = 1 - social[(social['country'] == country)
                            & (social['year'] == year)
                            ][d]
            else:
                val = social[(social['country'] == country)
                            & (social['year'] == year)
                            ][d]
            if not val.empty:
                ind.append(*val)
#         print(ind)
        social_ind[country].append(np.mean(ind))

for country in social_ind:
    social_years[country] = pd.Series(social_ind[country])
social_years.head()

C:\Users\3naza\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

C:\Users\3naza\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



,Year,Afghanistan,Albania,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bangladesh,...,Ukraine,United Kingdom,United States,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,2008,0.407580,NaN,NaN,0.241956,0.211878,0.605009,0.451359,0.406563,0.382699,...,0.117517,0.464084,0.456919,0.411628,NaN,0.296321,0.541987,NaN,0.357850,0.124358
1,2009,0.411005,NaN,NaN,0.224051,0.183155,NaN,NaN,0.352177,0.404583,...,0.090495,0.493190,0.463420,0.451405,NaN,0.297489,0.449857,0.324608,0.272062,0.178392
2,2010,0.332587,NaN,NaN,0.275407,0.190277,0.619424,0.489681,0.298425,0.380987,...,0.155693,0.537809,0.446322,0.479538,NaN,0.358110,0.483636,0.304377,NaN,0.311163
3,2011,0.311877,NaN,0.242311,0.373556,0.178276,0.612054,0.441654,0.338838,0.451400,...,0.168125,0.564048,0.423513,0.458217,NaN,0.319212,0.488588,0.274431,0.292913,0.292504
4,2012,0.359479,0.236043,0.165528,0.300296,0.172953,0.561572,0.408728,0.344851,0.395360,...,0.174731,0.561595,0.415171,0.473229,NaN,0.348617,0.447023,0.333302,0.393336,0.236262


In [164]:
suicides_shorten = shorten_data(suicides_years)
suicides_shorten['Social Index'] = \
pd.Series(shorten_data(social_years.fillna(0))['Sum'].values, index=suicides_shorten.index)

C:\Users\3naza\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [165]:
suicides_shorten['Sum'] /= max(suicides_shorten['Sum'])
suicides_shorten['Social Index'] /= max(suicides_shorten['Social Index'])
fig = px.scatter(suicides_shorten, x="Social Index", y="Sum", color='Year', trendline="ols",
                color_continuous_scale=px.colors.sequential.matter)
fig.show()

results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\3naza\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



C:\Users\3naza\Anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=11



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Sum   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                 -0.071
Method:                 Least Squares   F-statistic:                    0.3374
Date:                Mon, 09 Dec 2019   Prob (F-statistic):              0.576
Time:                        14:08:08   Log-Likelihood:                 18.568
No. Observations:                  11   AIC:                            -33.14
Df Residuals:                       9   BIC:                            -32.34
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9334      0.111      8.442      0.000       0.683       1.184
Social Index    -0.0905      0.156     -0.581      0.576      -0.443       0.262
==============================================================================
Omnibus:                       18.311   Durbin-Watson:                   2.206
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.200
Skew:                           1.924   Prob(JB):                      0.00224
Kurtosis:                       6.436   Cond. No.                         15.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [166]:
social_level = pd.DataFrame()
codes = []
countries = []
inds = []
for country in social_years.columns[1:]:
    try:
        codes.append(pycountry.countries.get(name=country).alpha_3)
        countries.append(country)
        inds.append(*social_years[social_years['Year'] == 2017][country])
    except:
        pass

social_level['Country Code'] = pd.Series(codes)
social_level['Country'] = pd.Series(countries)
social_level['Social Ind'] = pd.Series(inds)
social_level.head()

,Country Code,Country,Social Ind
0,AFG,Afghanistan,0.156864
1,ALB,Albania,0.324018
2,AGO,Angola,NaN
3,ARG,Argentina,0.277511
4,ARM,Armenia,0.215937


In [167]:
plot_world(social_level['Country Code'], social_level['Social Ind'], 'greens',
          social_level['Country'], "Social Index")

# Reason of crimes

In [168]:
import us_states_codes
# usa_shootings['State'] = pd.Series(abbrev2state[abbrev] for abbrev in usa_shootings['State'])
suicides = usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'] 

In [169]:
states = usa_shootings['State'].unique()
years = usa_shootings['Year'].unique()

usa_shootings_by_states = pd.DataFrame()
usa_shootings_by_states['Year'] = pd.Series(years)
for state in states:
    temp = []
    for year in years:
        temp.append(usa_shootings[(usa_shootings['State'] == state)
                                  & (usa_shootings['Year'] == year)
                                 ]['Total Injured/Killed Victims'].sum())
    usa_shootings_by_states[state] = pd.Series(temp)

usa_shootings_by_states.head()

,Year,DC,OH,CA,AR,DE,FL,UT,TX,TN,...,WY,RI,HI,NE,AK,SD,"St. Croix, US Virgin Islands",VT,ND,ME
0,1970,2,2,2,6,1,2,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,1971,0,1,4,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,1972,1,2,6,0,0,1,0,3,0,...,0,0,0,0,0,0,0,0,0,0
3,1973,0,5,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1974,0,0,7,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [170]:
usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'] = \
usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'].map({'Y': 'Y', 'N': 'N', 'N ': 'N',
                                                                     'Officer Involved': 'N',
                                                                     np.nan: 'U', 
                                                                      'Unknown': 'U'}, na_action='ignore')

In [171]:
by_states = pd.DataFrame()
states = []
codes = []
cases = []
values = []
killed = []
wounded = []
suicide = []
no_suicide = []
unknown_suicide = []

for state in usa_shootings_by_states.columns[1:]:
    if state in abbrev2state:
        states.append(abbrev2state[state])
        codes.append(state)
        values.append(usa_shootings_by_states[state].sum())
        cases.append(len(usa_shootings[usa_shootings['State'] == state]))
        killed.append(usa_shootings[usa_shootings['State'] == state]['Killed (includes shooter)'].sum())
        wounded.append(usa_shootings[usa_shootings['State'] == state]['Wounded'].sum())
        suicide.append(len(usa_shootings[ (usa_shootings['State'] == state) &\
                       (usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'] == 'Y')]\
                         ['Suicide (or attempted suicide) by Shooter (Y/N)']))
        no_suicide.append(len(usa_shootings[ (usa_shootings['State'] == state) &\
                       (usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'] == 'N')]\
                            ['Suicide (or attempted suicide) by Shooter (Y/N)']))
        unknown_suicide.append(len(usa_shootings[ (usa_shootings['State'] == state) &\
                       (usa_shootings['Suicide (or attempted suicide) by Shooter (Y/N)'] == 'U')]\
                                 ['Suicide (or attempted suicide) by Shooter (Y/N)']))

by_states['State'] = pd.Series(states).astype(str)
by_states['State Code'] = pd.Series(codes).astype(str)
by_states['Victims'] = pd.Series(values).astype(str)
by_states['Cases'] = pd.Series(cases).astype(str)
by_states['Killed'] = pd.Series(killed).astype(str)
by_states['Wounded'] = pd.Series(wounded).astype(int).astype(str)
by_states['Suicide (at lest attemption)'] = pd.Series(suicide).astype(str)
by_states['No Suicide'] = pd.Series(no_suicide).astype(str)
by_states['Unknown if Suicide'] = pd.Series(unknown_suicide).astype(str)

by_states.head()

,State,State Code,Victims,Cases,Killed,Wounded,Suicide (at lest attemption),No Suicide,Unknown if Suicide
0,District of Columbia,DC,28,25,13,15,0,25,0
1,Ohio,OH,75,58,18,57,4,54,0
2,California,CA,371,170,102,269,21,147,2
3,Arkansas,AR,39,18,11,28,4,14,0
4,Delaware,DE,9,9,4,5,1,8,0


In [172]:
def plot_usa(locations, z, colorscale, text, colorbar_title="", title_text=""):
    fig = go.Figure(data=go.Choropleth(
        locations = locations,
        z = z,
        locationmode = 'USA-states',
        colorscale = colorscale,
        autocolorscale=False,
        text=text,
        marker_line_color='white',
        colorbar_title = colorbar_title,
    ))

    fig.update_layout(
        title_text = title_text,
        geo = dict(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            ),
    )

    fig.show()

In [173]:
text = by_states['State'] + ' Cases: ' + by_states['Cases'] + '<br>' + 'Total victims: ' + by_states['Victims']\
        + ' Killed: ' + by_states['Killed'] + ' Wounded: ' + by_states['Wounded']\
        + '<br>Suicide of shooter (or attemption): ' + by_states['Suicide (at lest attemption)']\
        + '<br>No suicide: ' + by_states['No Suicide']\
        + ' Unknown if suicide: ' + by_states['Unknown if Suicide']

plot_usa(by_states['State Code'], by_states['Victims'], 'Reds', text,
         "Victims", 'School Shootings victims over all time')

In [174]:
weapon_laws = pd.read_csv(DATA_WEAPON + 'weapon_us.csv')
weapon_laws['State Code'] = weapon_laws['State'].map(state2abbrev)
weapon_laws.head()

,State,firearmRegRequired,permitReqToCarry,permitReqToPurchase,openCarryLegal,State Code
0,Alabama,False,True,False,True,AL
1,Alaska,False,False,False,True,AK
2,Arizona,False,False,False,True,AZ
3,Arkansas,False,False,False,True,AR
4,California,True,True,True,NaN,CA


In [175]:
plot_usa(weapon_laws['State Code'], weapon_laws['permitReqToPurchase'].map({True: 0, False: 1}),
        'purp', weapon_laws['State'], '', 'Where you can purchase a weapon without a permission')

In [176]:
shootings_weapons = pd.merge(by_states, weapon_laws, on='State')
ind = shootings_weapons[shootings_weapons['Cases'].astype(int) >= 8].index
cases_more_7 = pd.Series(np.nan, index=shootings_weapons.index)
cases_more_7.iloc[ind] = [1 for _ in range(len(ind))]
cases_more_7.fillna(0, inplace=True)
shootings_weapons['Cases>7'] = cases_more_7.astype(int)
shootings_weapons['Cases>7'].head()

0    1
1    1
2    1
3    1
4    1
Name: Cases>7, dtype: int32

In [177]:
shootings_weapons['permitReqToPurchase'].map({True: 0, False: 1}).corr(shootings_weapons['Cases>7'])

0.06675015657623448

In [178]:
shootings_weapons.head()

,State,State Code_x,Victims,Cases,Killed,Wounded,Suicide (at lest attemption),No Suicide,Unknown if Suicide,firearmRegRequired,permitReqToCarry,permitReqToPurchase,openCarryLegal,State Code_y,Cases>7
0,Ohio,OH,75,58,18,57,4,54,0,False,True,False,True,OH,1
1,California,CA,371,170,102,269,21,147,2,True,True,True,NaN,CA,1
2,Arkansas,AR,39,18,11,28,4,14,0,False,False,False,True,AR,1
3,Delaware,DE,9,9,4,5,1,8,0,False,True,False,True,DE,1
4,Florida,FL,137,90,56,81,14,75,1,False,True,False,False,FL,1


In [179]:
text = shootings_weapons['State'] + '<br>' + 'Permission is needed yo buy a weapon: '\
        + shootings_weapons['permitReqToPurchase'].astype(str) + '<br>'\
        + 'Cases: ' + shootings_weapons['Cases']\
        + ' Victims: ' + shootings_weapons['Victims']

plot_usa(shootings_weapons['State Code_x'],
        shootings_weapons['permitReqToPurchase'].map({True: 0, False: 1}) + shootings_weapons['Cases>7'],
        'purp', text)

Dark purple means that in the state you can purchase a weapon without permission and there were more than 7 cases of school shootings. Light purple means only one of these statements.

As we saw games' sold are measured in millions of copies. So, it can also have a good impact on economy. Unfortunately, I didn't found prepared data about this, so the numbers below are copied and pasted here one by one by my hands.

Tax of vide game industry:<br>
In 2013: 10 931 922 690 dollars., total USA GDP: 16.6915 trillions<br>
In 2015: 11 571 700 010 dollars., total USA GDP: 18.1207 trillions


In [180]:
industry = ['Video Game Industry', 'Manufacturing', 'Retail Trade', 'Finance and Insurance',
            'Agriculture, Forestry, Fishing, and Hunting', 'Food and Beverage and Tobacco Products Manufacturing',
           'Textile Mills and Textile Product Mills', 'Construction', 'Real Estate']
annual_growth = [3.68, 1.25, 3.24, 2.32, .9, -.95, 1.5, 3.28, 1.84]

In [181]:
Industry = ['Video Game Industry', 'Advertising and Related Services',
            'Custom Computer Programming Services',
            'Computer and Electronic Product Manufacturing', 'Aerospace Product and Parts Manufacturing',
            'Newspaper Publishers', 'Food Manufacturing', 'Textile Mills', 'Chemical Manufacturing',
            'Total US employments']
employment_2013 = [57, 452, 739, 1066, 495, 213, 1474, 117, 793, 136381]
employment_2015 = [60, 484, 829, 1050, 488, 191, 1505, 116, 810, 141865]
growth_rate = [2.88, 3.46, 5.95, -.74, -.78, -5.27, 1.05, -.34, 1.06, 1.99]
employment_df = pd.DataFrame()
employment_df['Industry'] = pd.Series(Industry)
employment_df['Employment 2013'] = pd.Series(employment_2013) * 1000
employment_df['Employment 2015'] = pd.Series(employment_2015) * 1000
employment_df['Annual growth rate'] = pd.Series(growth_rate)

fig = go.Figure(data=[go.Pie(labels=employment_df['Industry'].values[:-1],
                             values=employment_df['Employment 2015'].values[:-1])])
fig.update_layout(title_text='Some of US employments in 2015')
fig.show()

About 60K employed Americans is a big number. And unlike other industries, game industry is growing fast.

In [186]:
fig = go.Figure()
fig.add_trace(go.Bar(x=employment_df['Industry'],
                     y=employment_df['Annual growth rate'],
                    marker_color='limegreen',
                    name='expenses'))
fig.update_layout(title_text='Annual employment rate growth in %')
fig.show()

# Conclusions

We cannot definitely say how computer games influence on our minds, but here are some things that are more or less associated with video games.
 - higher grades among boys
 - lower alcohol consumption, no history of depression, getting into serious fights and carrying a weapon among girls
 - lower smoking and marijuana usage
 - higher caffeine consumption
 - lower school shootings rate$^s$
 - growth of employment rate
 
*s* - it's not totaly clear, as we don't know who of shooters played computer games